<a href="https://colab.research.google.com/github/Medynal/thing/blob/main/PRAC_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Brownfield Redevelopment Tax Credits are incentives given to encourage the clean-up and redevelopment of brownfields i.e. properties that are abandoned or underutilized due to contamination or perceived contamination.
The Department of Taxation and Finance annually produces a mandated report of taxpayers claiming Brownfield Redevelopment credit to offset their tax. The aim of this analysis is to explore the policy impact in New York City from the time the policy was implemented till 2024 (does the program improve the number of brownfields developed annually in New York City(2007-2024)?


In [578]:
#Importing raw data
!pip -q install gdown==4.6.0
import gdown
gdrive_link = 'https://drive.google.com/file/d/1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E/view?usp=sharing'
file_id= '1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E'
downloadable_link= f'https://drive.google.com/uc?id={file_id}'
gdown.download(downloadable_link, output= 'NYC_brownfields.csv')
print ('download complete!' )

Downloading...
From: https://drive.google.com/uc?id=1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E
To: /content/NYC_brownfields.csv
100%|██████████| 237k/237k [00:00<00:00, 74.2MB/s]

download complete!


This file import method was adopted to enable easy assessibilty of the raw data by future project user. The google drive link is set to public.

In [579]:
# Importing necessary frameworks
import pandas as pd
import numpy as np
import re

brownfield_df= pd.read_csv('NYC_brownfields.csv')
print(f'shape: {brownfield_df.shape}\n') #checking number of rows and columns
print(f'Missin Values:\n {brownfield_df.isna().sum()}')# Checking for missing values in each column
print('\nData Information:') # General Information
brownfield_df.info()

shape: (1461, 19)

Missin Values:
 Calendar Year                                0
Tax Year                                     0
Taxpayer Name                                0
Project Site Code                            0
Project Site Name                            0
Notes                                      389
DEC Region                                   0
County                                       0
Municipality                                 0
En-Zone                                      0
 Site Preparation Component Costs            1
Site Preparation Component Credit            1
 Tangible Property Component Costs           3
Tangible Property Component Credit           1
 OnSite Groundwater Remediation Cost         3
 OnSite Groundwater Remediation Credit       5
 Total Cost                                  1
 Total Credit                                0
Georeference                                 3
dtype: int64

Data Information:
<class 'pandas.core.frame.DataFrame'>
Ra

1. Calendar year: Calendar year in which the tax return was filed
2. Tax Year: Tax year of the credit claim; typically the year preceding the calendar year, although extensions and fiscal years may result in claims for a tax year being filed several calendar years later.
3. Taxpayer Name: Name of the entity earning the credit.
4. Project Site Code: Brownfield Certificate of Completion (COC) code number per the Department of Environmental Conservaction.  
5. Project Site Name: Brownfield Certificate of Completion (COC) number and site name per the Department of Environmental Conservation (DEC).
6. Notes:Remarks about the project or credit
7. DEC Region:1-9 depending on site location; code assigned by Department of Environmental Conservation corresponding to regions within NYS.
8. County:County Name
9. Municipality:City, town, borough, county, or village name where project is located For the location parameters and site description
10. En-Zone:Is site located in an Environmental Zone - Yes/No An environmental zone (EN-Zone) is an area designated as such by the Commissioner of Labor. Such areas shall be census tracts that satisfy certain poverty and unemployment metrics.
11. Site Preparation Component Costs: Generally, costs related to the remediation of a brownfield site (cleanup costs). Eligible costs differ depending on a sites date of acceptance in to the Brownfield Cleanup Program.
12. Site Preparation Component Credit: The product of the eligible site preparation costs for the qualified site multiplied by the applicable percentage.
13. Tangible Property Component Costs: Generally, costs related to the redevelopment, such as construction of a building, occurring on a brownfield site. Eligible costs differ depending on a sites date of acceptance in to the Brownfield Cleanup Program.
14. Tangible Property Component Credit: The product of the cost of qualified tangible property multiplied by the applicable percentage. The credit is also subject to varying caps dependent upon a site’s use and the amount of cleanup costs.
15. OnSite Groundwater Remediation Cost:Generally, costs related to the remediation of groundwater on a brownfield site (cleanup costs). Eligible costs differ depending on a sites date of acceptance in to the Brownfield Cleanup Program.
16. OnSite Groundwater Remediation Credit:The product of the on-site groundwater remediation costs multiplied by the applicable percentage.
17. Total Cost:The sum of component costs for the Site Preparation Credit, the Tangible Property Credit, and the OnSite Groundwater Remediation Credit.
18. Total Credit: The sum of component credits for the Site Preparation Credit, the Tangible Property Credit, and the OnSite Groundwater Remediation Credit.
19. Georeference: Open Data/Socrata-generated geocoding information from supplied address components.

Source:https://data.ny.gov/Government-Finance/Brownfield-Redevelopment-Credit-Beginning-Calendar/vud8-75x8/about_data

In [580]:
#rename columns with long names for easy access.
brownfield_df.columns = brownfield_df.columns.str.strip()
brownfield_df.rename(columns={'Site Preparation Component Costs':'spc_cost',
                              'Site Preparation Component Credit':'spc_credit',
                              'Tangible Property Component Costs':'tpc_cost',
                              'Tangible Property Component Credit':'tpc_credit',
                              'OnSite Groundwater Remediation Cost':'ogrc_cost',
                              'OnSite Groundwater Remediation Credit':'ogrc_credit'},inplace=True)
# Change all colmn names to lower case and replace white space with '_'
brownfield_df.columns= brownfield_df.columns.str.lower().str.replace(' ', '_')

Assumption: Most credit awarded are half of the cost accrued. therefore, it is assumed that:
Missing credit is half of the cost if available
Missing cost is twice the credit, if available
If both are missing, then both are zero.

For missing geospatial reference, it is assumed that project with the same project site code will have the same georeference code

In [581]:
#Data Cleaning
## dealing with mising values for cost and credit
def na_values(row, a, b): #define a function that returns half of cost if credit is missing and double the credit if course is missing
    if pd.isna(row[a]) and pd.notna(row[b]):
        row[a] = row[b]*2
    elif pd.notna(row[a]) and pd.isna(row[b]):
        row[b] = row[a]/2
    elif pd.isna(row[a]) and pd.isna(row[b]):
        row[a], row[b] = 0, 0
    return row
brownfield_df = brownfield_df.apply(na_values, axis=1, args=('spc_cost', 'spc_credit'))
brownfield_df = brownfield_df.apply(na_values, axis=1, args=('tpc_cost', 'tpc_credit'))
brownfield_df = brownfield_df.apply(na_values, axis=1, args=('ogrc_cost', 'ogrc_credit'))
#filling missing total cost and total credit
total_cost = brownfield_df['spc_cost'] + brownfield_df['tpc_cost'] + brownfield_df['ogrc_cost']
total_credit = brownfield_df['spc_credit'] + brownfield_df['tpc_credit'] + brownfield_df['ogrc_credit']
brownfield_df['total_cost']= brownfield_df['total_cost'].fillna(total_cost)
##recalculating total cost and total credit due to fixed missing values
brownfield_df['total_cost']= total_cost
brownfield_df['total_credit']= total_credit


Rows with missing georeference value has the same project site code (C622031). other rows with the same project site code has a georeference of POINT (-74.98433 43.0259)


In [582]:
## show rows with missing georefernce
brownfield_df[brownfield_df['georeference'].isna()]
##check for complete rows with the same project site code
brownfield_df[brownfield_df['project_site_code']== 'C622031']
brownfield_df['georeference']= brownfield_df['georeference'].fillna('POINT (-74.98433 43.0259)')
#fill missing notes
brownfield_df['notes']= brownfield_df['notes'].fillna('No notes available')


Region code:
1 = Long Island
2 = New York City
3 = Lower Hudson Valley
4 = Capital Region/Northern Catskills
5 = Eastern Adirondacks/Lake Champlain
6 = Western Adirondacks/Eastern Lake Ontario
7 = Central New York
8 = Western Finger Lakes
9 = Western New York

Source:https://data.ny.gov/Government-Finance/Brownfield-Redevelopment-Credit-Beginning-Calendar/vud8-75x8/about_data


In [583]:
#replacing region code with region names
region_dict= {1:'Long Island',
                2:'New York City',
                3:'Lower Hudson Valley',
                4:'Capital Region/Norther Catskills',
                5:'Eastern Adirondacks/Lake Champlain',
                6:'Western Adirondacks/Eastern Lake Ontario',
                7:'Central New York',
                8:'Western Finger Lakes',
                9:'Western New York'}
brownfield_df['dec_region']= brownfield_df['dec_region'].astype(int).map(region_dict)

Notes:

1/ Information from amended return that supersedes a previously filed return; original filing removed from public report
2/ Credit form not filed so no cost data available; credit data taken from main form; site information taken from a pass-through entity
3/ The Department of Environmental Conservation (DEC) accepted this site into the Brownfield Cleanup Program on or after June 23, 2008, but prior to July 1, 2015
4/ The Department of Environmental Conservation (DEC) accepted this site into the Brownfield Cleanup Program on or after July 1, 2015
5/ Qualified site located in a Brownfield Opportunity Area
6/ Negative amounts indicate credit recapture
7/ Qualified site developed as an affordable housing project 8/ Combined entry of multiple sites. Total credit also includes credit from site C828125 River Park Commons-Townhouses. It is not possible to do site-specific costs/credit on how the credit was claimed in the aggregate

source: https://data.ny.gov/Government-Finance/Brownfield-Redevelopment-Credit-Beginning-Calendar/vud8-75x8/about_data

In [584]:
from typing_extensions import Pattern
# replacing notes code with actual notes
notes_dict= {'1/':'Information from amended return that supersedes a previously filed return; original filing removed from public report',
'2/':'Credit form not filed so no cost data available; credit data taken from main form; site information taken from a pass-through entity',
'3/': 'The Department of Environmental Conservation (DEC) accepted this site into the Brownfield Cleanup Program on or after June 23, 2008, but prior to July 1, 2015',
'4/': 'The Department of Environmental Conservation (DEC) accepted this site into the Brownfield Cleanup Program on or after July 1, 2015',
'5/': 'Qualified site located in a Brownfield Opportunity Area',
'6/': 'Negative amounts indicate credit recapture',
'7/': 'Qualified site developed as an affordable housing project',
'8/': 'Combined entry of multiple sites. Total credit also includes credit from site C828125 River Park Commons-Townhouses. It is not possible to do site-specific costs/credit on how the credit was claimed in the aggregate'
}
def note(a):
  a = str(a).strip()
  matches = re.findall(r'\d+/', a)
  if matches:# Look up each note code in the dictionary and collect their meanings
    notes = [notes_dict.get(m, 'Not Available') for m in matches]
    return ', '.join(notes)
  else:
    return 'No Notes'

# Apply function notes column
brownfield_df['notes'] = brownfield_df['notes'].apply(note)







In [585]:
##recheck for missing values
brownfield_df

,calendar_year,tax_year,taxpayer_name,project_site_code,project_site_name,notes,dec_region,county,municipality,en-zone,spc_cost,spc_credit,tpc_cost,tpc_credit,ogrc_cost,ogrc_credit,total_cost,total_credit,georeference
0,2022,2021,NEW 470 LLC,C224242,New 470 Project,The Department of Environmental Conservation (...,New York City,Kings,Brooklyn,No,0.0,0.0,103861.0,15579.0,0.0,0.0,103861.0,15579.0,POINT (-73.99036 40.69245)
1,2009,2008,EMPIRE GEN HOLDINGS INC,C442035,South 40 Site,No Notes,Capital Region/Norther Catskills,Rensselaer,Rensselaer,No,700000.0,84000.0,0.0,0.0,0.0,0.0,700000.0,84000.0,POINT (-73.74127 42.64472)
2,2020,2019,NEW 470 LLC,C224242,New 470 Project,The Department of Environmental Conservation (...,New York City,Kings,Brooklyn,No,2935248.0,821869.0,0.0,0.0,0.0,0.0,2935248.0,821869.0,POINT (-73.99036 40.69245)
3,2018,2017,BLA II LLC,C915262,125 Main Street Site,The Department of Environmental Conservation (...,Western New York,Erie,Buffalo,No,0.0,0.0,43537.0,4354.0,0.0,0.0,43537.0,4354.0,POINT (-78.87846 42.88545)
4,2016,2015,BLA II LLC,C915262,125 Main Street Site,The Department of Environmental Conservation (...,Western New York,Erie,Buffalo,No,0.0,0.0,32274.0,3227.0,0.0,0.0,32274.0,3227.0,POINT (-78.87846 42.88545)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2018,2017,SENECA MARKET I LLC,C849004,Seneca Market I LLC,No Notes,Western Finger Lakes,Schuyler,Watkins Glen,Yes,0.0,0.0,10768.0,1077.0,0.0,0.0,10768.0,1077.0,POINT (-76.87373 42.38107)
1457,2024,2023,MAIN & DODGE LLC,C915341,1155 Main Street,The Department of Environmental Conservation (...,Western New York,Erie,Buffalo,Yes,0.0,0.0,67819.0,13564.0,0.0,0.0,67819.0,13564.0,POINT (-78.87846 42.88545)
1458,2017,2016,ZURENDA ENTERPRISES INC,C704045,TCMF Hillcrest Facility,No Notes,Central New York,Broome,Binghamton,No,0.0,0.0,0.0,0.0,68349.0,8202.0,68349.0,8202.0,POINT (-75.91129 42.09871)
1459,2009,2008,SENECA MARKET I LLC,C849004,Seneca Market I LLC,No Notes,Western Finger Lakes,Schuyler,Watkins Glen,Yes,1368788.0,136878.0,16040015.0,1604002.0,0.0,0.0,17408803.0,1740880.0,POINT (-76.87373 42.38107)


In [586]:
## split the georefernce to longitude and latitude
def coordinate(g):
  g= g.strip()
  pattern = r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)'
  match = re.search(pattern, g)
  if match:
    longitude = float(match.group(1))
    latitude = float(match.group(2))
    return pd.Series([longitude, latitude])
  else:
    return pd.Series([None, None])

brownfield_df[['longitude', 'latitude']] = brownfield_df['georeference'].apply(coordinate)


In [587]:
## checking for missing values===> No missing values!
brownfield_df[brownfield_df['longitude'].isna()]


,calendar_year,tax_year,taxpayer_name,project_site_code,project_site_name,notes,dec_region,county,municipality,en-zone,...,spc_credit,tpc_cost,tpc_credit,ogrc_cost,ogrc_credit,total_cost,total_credit,georeference,longitude,latitude


In [588]:
brownfield_dffinal= brownfield_df.copy()
## checking final data details
print(f'shape: {brownfield_dffinal.shape}\n')
print(f'Missin Values:\n {brownfield_dffinal.isna().sum()}')
print('\nData Information:')
brownfield_dffinal.info()
print('\nNumber of Unique Values in each column:')
print(brownfield_dffinal.nunique())



shape: (1461, 21)

Missin Values:
 calendar_year        0
tax_year             0
taxpayer_name        0
project_site_code    0
project_site_name    0
notes                0
dec_region           0
county               0
municipality         0
en-zone              0
spc_cost             0
spc_credit           0
tpc_cost             0
tpc_credit           0
ogrc_cost            0
ogrc_credit          0
total_cost           0
total_credit         0
georeference         0
longitude            0
latitude             0
dtype: int64

Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   calendar_year      1461 non-null   int64  
 1   tax_year           1461 non-null   int64  
 2   taxpayer_name      1461 non-null   object 
 3   project_site_code  1461 non-null   object 
 4   project_site_name  1461 non-null   object 
 5   no

In [589]:
##Save cleaned data
brownfield_dffinal.to_csv('NYC_brownfields_cleaned.csv', index=False)

Descriptive Analysis (Get to Know the Data)
In this section, the following questions will be answered to give overview of the data.

Project trend over time:How many redevelopment projects happened each year?

Total Costs vs. Credits Over Time:Are redevelopment costs and credits increasing or decreasing over the years?

Top Taxpayers/Developers: Which companies received the most credits or completed the most projects?

Regional Distribution:Which counties or DEC regions have the most brownfield projects?

In [590]:
# import necessary frameworks
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
pio.templates.default = "ggplot2"


In [591]:
print('Overview of Brownfield Redevelopment policy from 2007 to 2024\n')
print(f'Total Number of Brownfield Developed: {brownfield_dffinal['project_site_code'].nunique()}')
print(f'Total Number of Developers: {brownfield_dffinal["taxpayer_name"].nunique()}')
print(f'Total Number of DEC Regions in NYC: {brownfield_dffinal["dec_region"].nunique()}')
print(f'Total Number of Counties in NYC: {brownfield_dffinal["county"].nunique()}')
print(f'Total Number of Municipalities in NYC: {brownfield_dffinal["municipality"].nunique()}')
print(f'Total cost of Brownfields project: ${brownfield_dffinal["total_cost"].sum():,}')
print(f'Total amount spent on groundwater remediation: ${brownfield_dffinal["ogrc_cost"].sum():,}')
print(f'Total amount spent on site preparation: ${brownfield_dffinal["spc_cost"].sum():,}')
print(f'Total amount spent on tangible property: ${brownfield_dffinal["tpc_cost"].sum():,}')
print(f'Total credit awarded: ${brownfield_dffinal["total_credit"].sum():,}')

Overview of Brownfield Redevelopment policy from 2007 to 2024

Total Number of Brownfield Developed: 530
Total Number of Developers: 605
Total Number of DEC Regions in NYC: 9
Total Number of Counties in NYC: 41
Total Number of Municipalities in NYC: 119
Total cost of Brownfields project: $31,820,206,255.0
Total amount spent on groundwater remediation: $121,322,612.0
Total amount spent on site preparation: $3,726,441,039.0
Total amount spent on tangible property: $27,972,442,604.0
Total credit awarded: $5,028,306,117.0


In [592]:
#Data Visualisation
##Project trend over time:How many redevelopment projects happened each year?
project_trend= brownfield_dffinal.groupby('calendar_year')['project_site_code'].nunique().reset_index(name= 'total_project')
#print(project_trend.head(3))
trend= px.line(data_frame= project_trend,
                       x= 'calendar_year',
                       y= 'total_project',
                       color_discrete_sequence=['#bf5824'],
                       title= 'Number of Brownfields Developed Anually',
                       labels= {'calendar_year': 'Calendar Year', 'total_project':'Total Brownfeilds Developed'},
                      hover_name= 'calendar_year',
                      markers=True,
                      height= 700)

trend.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)
trend.show()
'''Fig1.1:Number of Brownfields Developed Anually'''

'Fig1.1:Number of Brownfields Developed Anually'

In [593]:
#Total Cost Vs Total Credit over time
total_cost_credit= brownfield_dffinal.groupby('calendar_year')[['total_cost', 'total_credit']].sum().reset_index()
total_cost_credit= total_cost_credit.melt(id_vars= 'calendar_year', value_vars= ['total_cost', 'total_credit'], var_name= 'type', value_name= 'amount')
total_bar= px.bar(data_frame= total_cost_credit,
                       x= 'calendar_year',
                       y= 'amount',
                       color= 'type',
                       barmode= 'group',
                       color_discrete_map={'total_cost':'#2A9D8F', 'total_credit':'#bf5824'},
                       labels={'calendar_year':'Calendar Year', 'amount':'Amount', 'type':'Type'},
                       title= 'Total Cost Vs Total Credit over time')
total_bar.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)
total_bar.show()
'''Fig1.2:Total Cost Vs Total Credit over time'''


'Fig1.2:Total Cost Vs Total Credit over time'

Top Taxpayers/Developers – Which companies received the most credits or completed the most projects?

In [594]:
#Top taxpayer by project developed
taxpayer_counts = brownfield_dffinal.groupby(['calendar_year', 'taxpayer_name'])['project_site_code'].nunique().reset_index(name='Number_of_projects')
##Find the top taxpayer per year
top_taxpayer = taxpayer_counts.loc[taxpayer_counts.groupby('calendar_year')['Number_of_projects'].idxmax()].reset_index(drop=True)
top_taxpayer.head(3)
top_taxpayer_bar= px.bar(data_frame= top_taxpayer,
                       y= 'calendar_year',
                       x= 'Number_of_projects',
                       orientation= 'h',
                       color= 'taxpayer_name',
                       height= 800,
                       color_discrete_sequence= px.colors.qualitative.Pastel,
                       text= 'Number_of_projects',
                       labels={'calendar_year':'Calendar Year', 'Number_of_projects':'Number of Brownfields Developed', 'taxpayer_name':'Taxpayer Name'},
                       title= 'Annual Top Taxpayer by project developed')

top_taxpayer_bar.update_yaxes(tickangle=45, tick0= 2007, dtick= 1)
top_taxpayer_bar.show()
'''Fig1.3:Annual Top Taxpayer by project developed'''

'Fig1.3:Annual Top Taxpayer by project developed'

In [595]:
#Top taxpayer by project credit
taxpayer_ccounts = brownfield_dffinal.groupby(['calendar_year', 'taxpayer_name'])['total_credit'].sum().reset_index()
##Find the top taxpayer per year
topc_taxpayer = taxpayer_ccounts.loc[taxpayer_ccounts.groupby('calendar_year')['total_credit'].idxmax()].reset_index(drop=True)
topc_taxpayer.head(3)
topc_taxpayer_bar= px.bar(data_frame= topc_taxpayer,
                       x= 'calendar_year',
                       y= 'total_credit',
                       color= 'taxpayer_name',
                       color_discrete_sequence= px.colors.qualitative.Pastel,
                       labels={'calendar_year':'Calendar Year', 'total_credit':'Total Credit Earned', 'taxpayer_name':'Taxpayer Name'},
                       title= 'Top Taxpayer by Total Credit Earned')

topc_taxpayer_bar.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)
topc_taxpayer_bar.update_traces(hovertemplate='%{y:,}')
topc_taxpayer_bar.show()
'''Fig1.4:Annual Top Taxpayer by Total Credit Earned'''

'Fig1.4:Annual Top Taxpayer by Total Credit Earned'

In [596]:
## Number of Brownfields Developed so far by DEC Region
region_trend= brownfield_dffinal.groupby(['dec_region', 'en-zone'])['project_site_code'].nunique().reset_index()
region_trend_bar= px.bar(data_frame= region_trend,
                       x= 'dec_region',
                       y= 'project_site_code',
                       color= 'en-zone',
                       barmode= 'relative',
                       color_discrete_map={'No':'#2A9D8F', 'Yes':'#bf5824'},
                       text= 'project_site_code',
                       labels={'dec_region':'Department of Environmental Conservation region ', 'project_site_code':'Number of Brownfields Developed', 'en-zone':'Environmental Zone'},
                       title= 'Total Brownfields Developed in each DEC region based on Environment Zone')
region_trend_bar.show()
'''Fig1.5:Total of Brownfields Developed in each DEC region based on Environment Zone'''


'Fig1.5:Total of Brownfields Developed in each DEC region based on Environment Zone'

In [597]:
#Number of Brownfields Developed so far by DEC Region
import random
random.seed(59)
custom_colors = [f'#{random.randint(0, 0xFFFFFF):06x}' for County in range(brownfield_dffinal['county'].nunique())]# generate distinct color
fig = px.scatter_mapbox(brownfield_dffinal,
    lat='latitude',
    lon='longitude',
    hover_name='project_site_name',
    hover_data=['county', 'calendar_year'],
    color='county',
    zoom=5,
    height=600,
    title='Brownfield Redevelopment Project Locations (NY)',
    color_discrete_sequence= custom_colors,
    mapbox_style= 'carto-positron'
)
fig.show()


In [598]:
# Top ten counties base on brownfield developed
top_ten_counties= pd.pivot_table(brownfield_dffinal,
                                 index= 'county', columns= 'en-zone',values= 'project_site_code',
                                 fill_value=0, aggfunc= 'nunique').reset_index()
top_ten_counties['total projects']= top_ten_counties['Yes']+top_ten_counties['No']
top_ten_counties= top_ten_counties.sort_values(by= 'total projects', ascending= False).head(10)
#.reset_index()
top_ten_data= top_ten_counties.melt(id_vars= 'county', value_vars= ['Yes', 'No'], var_name= 'en-zone', value_name= 'brownfields')
top_ten_bar= px.bar(data_frame= top_ten_data,
                       x= 'brownfields',
                       y= 'county',
                       color= 'en-zone',
                       barmode= 'relative',
                       color_discrete_map={'No':'#2A9D8F', 'Yes':'#bf5824'},
                       text= 'brownfields',
                       labels={'brownfieds':'Number of Brownfields Developed (2007-2024) ', 'county':'County', 'en-zone':'Environmental Zone'},
                       title= 'Top ten counties base on brownfield developed')
top_ten_bar.show()

'''Fig1.6:Top ten counties base on brownfield developed'''




'Fig1.6:Top ten counties base on brownfield developed'

2.ENVIRONMENTAL ANALYSIS:analyze the impact and efficiency of redevelopment efforts:

Projects in “En-Zones” vs. non-En-Zones: Are environmental zones getting more redevelopment attention?

Groundwater Remediation Trends: Which areas spend the most on cleaning up groundwater?

Cleanup vs. Construction: Compare investment in environmental cleanup vs. property construction.

In [599]:
#Trend of brownfields developed anually based on environment zone
zone_trend= brownfield_dffinal.groupby(['calendar_year','en-zone'])['project_site_code'].nunique().reset_index()
#print(zone_trend)
zone_trend_bar= px.bar(data_frame= zone_trend,
                       x= 'calendar_year',
                       y= 'project_site_code',
                       color= 'en-zone',
                       barmode= 'group',
                       color_discrete_map={'No':'#2A9D8F', 'Yes':'#bf5824'},
                       text= 'project_site_code',
                       labels={'calendar_year':'Calendar Year', 'project_site_code':'Number of Brownfields Developed', 'en-zone':'Environmental Zone'},
                       title= 'Trend of Brownfields Developed Anually based on Environment Zone')
zone_trend_bar.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)
zone_trend_bar.show()





In [600]:
# Top ten counties base on brownfield developed
top_ten_counties= pd.pivot_table(brownfield_dffinal,
                                 index= 'county', columns= 'en-zone',values= 'project_site_code',
                                 fill_value=0, aggfunc= 'nunique').reset_index()
top_ten_counties['total projects']= top_ten_counties['Yes']+top_ten_counties['No']
top_ten_counties= top_ten_counties.sort_values(by= 'total projects', ascending= False).head(10)
#.reset_index()
top_ten_data= top_ten_counties.melt(id_vars= 'county', value_vars= ['Yes', 'No'], var_name= 'en-zone', value_name= 'brownfields')
top_ten_bar= px.bar(data_frame= top_ten_data,
                       x= 'brownfields',
                       y= 'county',
                       color= 'en-zone',
                       barmode= 'relative',
                       color_discrete_map={'No':'#2A9D8F', 'Yes':'#bf5824'},
                       text= 'brownfields',
                       labels={'brownfieds':'Number of Brownfields Developed (2007-2024) ', 'county':'County', 'en-zone':'Environmental Zone'},
                       title= 'Top ten counties base on brownfield developed')
top_ten_bar.show()




In [601]:
#annual ground water remediation cost by each dec region
brownfield_distinct= brownfield_dffinal.groupby(['calendar_year','project_site_code','dec_region','county', 'municipality','project_site_name' ])['ogrc_cost'].sum().reset_index()
brownfield_distinct.head(3)
'''ogrc= px.scatter(data_frame= brownfield_dffinal,
                 x= 'calendar_year',
                 y= 'ogrc_cost',
                 color= 'dec_region',
                 color_discrete_sequence= custom_colors,
                 title= 'Brownfield Ground Water Remediation Cost',
                 hover_name= 'project_site_code',
                 hover_data= ['county', 'municipality'],
                 labels= {'ogrc_cost':'Ground Water Remediation Cost', 'calendar_year': 'Calender Year' })
ogrc.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)
ogrc.update_yaxes(tickprefix= '$', tick)
ogrc.update_traces(marker=dict(size=10))
ogrc.show()'''


"ogrc= px.scatter(data_frame= brownfield_dffinal, \n                 x= 'calendar_year',\n                 y= 'ogrc_cost', \n                 color= 'dec_region', \n                 color_discrete_sequence= custom_colors,\n                 title= 'Brownfield Ground Water Remediation Cost',\n                 hover_name= 'project_site_code',\n                 hover_data= ['county', 'municipality'],\n                 labels= {'ogrc_cost':'Ground Water Remediation Cost', 'calendar_year': 'Calender Year' })\nogrc.update_xaxes(tickangle=90, tick0= 2007, dtick= 1)\nogrc.update_yaxes(tickprefix= '$', tick)\nogrc.update_traces(marker=dict(size=10))\nogrc.show()"